# Clean and compile raw data

In this first piece of code, we read in the raw Washington and California higher education public employee earnings data and Bureau of Economic Analysis state price parities data. We clean and format these datasets, and save them out for use in `1_analyze_data.ipynb`.


First, we will import all the necessary packages we will work with.

In [1]:
rm(list = ls())
require(data.table)
require(stringr)
require(readxl)

Loading required package: data.table

Loading required package: stringr

Loading required package: readxl



We will start by processing the Washington state earnings data. This is pulled in from two separate files, one containing 2010-2013 data, and another for 2014-2018. We format the columns and filter to keep only higher education employer institutions.

In [2]:
# Process 2010-2013 data
wa_salaries_2010_2013 <- fread("../data_raw/WA_Annual_Salary_2010_2013.csv")
setnames(wa_salaries_2010_2013, c("Agency", "AgencyTitle", "Salary2010", "Salary2011", "Salary2012", "Salary2013"), 
         c("Agy", "AgyTitle", "Sal2010", "Sal2011", "Sal2012", "Sal2013"))
# Process 2014-2018 data. Note that this dataset is saved out in two separate files due to Github's file size limit
wa_salaries_2014_2018_1 <- fread("./WA_Annual_Salary_2014_2018_1.csv")
wa_salaries_2014_2018_2 <- fread("./WA_Annual_Salary_2014_2018_2.csv")
wa_salaries_2014_2018 <- rbind(wa_salaries_2014_2018_1, wa_salaries_2014_2018_2)

# Bind together all years of data, and format columns
WA_salaries <- rbind(wa_salaries_2010_2013, wa_salaries_2014_2018, fill = T)
WA_salaries <- melt(WA_salaries, id.vars = c("Agy", "AgyTitle", "JobTitle"), 
                      variable.name = "year", value.name = "salary")
WA_salaries <- WA_salaries[, year := str_replace(year, "Sal", "")]
WA_salaries <- WA_salaries[, year := as.numeric(as.character(year))]
WA_salaries <- WA_salaries[, .(year, employer = AgyTitle, title = JobTitle, salary, state = "WA")]
  
# Filter data to only higher education institutions
WA_salaries <- WA_salaries[grepl("College", employer) | grepl("University", employer)]

Next, we process the California state earnings data. This data is stored as separate files for each year and each employer agency, so we loop over all the files to read them in, and filter and format the relevant columns.

In [3]:
# Create lists of all file names containing California earnings data
csu_files <- list.files(path = "../data_raw/", pattern = "*_CaliforniaStateUniversity.csv", full.names = TRUE)
uc_files <- list.files(path = "../data_raw/", pattern = "*_UniversityOfCalifornia.csv", full.names = TRUE)
colleges_files <- list.files(path = "../data_raw/", pattern = "*_CommunityCollegeDistrict.csv", full.names = TRUE)
all_files <- c(csu_files, uc_files, colleges_files)
  
# Loop over all file names and append together
CA_salaries <- fread(all_files[1])
for (i in 2:length(all_files)) {
    dt <- fread(all_files[i])
    CA_salaries <- rbind(CA_salaries, dt, fill = T)
}

# Fill in specific names for community colleges
CA_salaries[EmployerType == "Community College District", EmployerType := EmployerName]

  
# Format columns  
# Note the two job title fields; we append these together in order to get more detailed information
CA_salaries <- CA_salaries[, .(year = Year, employer = EmployerType, 
                                 title = paste(Position, ";", OtherPositions), salary = TotalWages, state = "CA")]

# Fix encoded characters in job titles so we can uppercase them
CA_salaries[, title := toupper(iconv(title, from = "latin1", to = "UTF-8"))]

Finally, append the Washington and California data and save out a cleaned file containing only 2018 data, due to Github's file size limits.

In [4]:
salaries <- rbind(WA_salaries, CA_salaries, fill = T)
# Drop missing salary data
salaries <- salaries[!(is.na(salary)) & salary > 0]
  
# Format job titles
salaries[, title := gsub("[.]","",title)]
salaries[, title := gsub("-","",title)]
salaries <- salaries[, title := paste0(" ", title, " ")]
  
fwrite(salaries[year == 2018], "../data_clean/edu_salaries.csv")

Next, we read in state all-sector purchasing price parities data in two separate files containing 2017 and 2008-2016, format the data, append, and save out. Note that we don't have data through 2018 so we assume that 2018 values are equal to 2017 values.

In [5]:
# Process 2017 data
ppp_2017 <- fread("../data_raw/SARPP_STATE_2017.csv")[, 1:2]
ppp_2017 <- ppp_2017[V1 %in% c("California", "Washington")]  
ppp_2017 <- ppp_2017[, .(state = V1, year = 2017, adj = as.integer(V2) / 100)]
 
# Process 2008-2016 data
ppp_2008_2016 <- fread("../data_raw/SARPP_STATE_2008_2016.csv")
ppp_2008_2016 <- ppp_2008_2016[Description == "RPPs: All items" & GeoName %in% c("California", "Washington")]
ppp_2008_2016 <- melt(ppp_2008_2016, id.vars = c("GeoName"), variable.name = "year")
ppp_2008_2016 <- ppp_2008_2016[19:32]
ppp_2008_2016 <- ppp_2008_2016[, .(state = GeoName, year = as.integer(as.character(year)), adj = as.integer(value) / 100)]
  
# Bind together all years, adding an extra 2018 row
ppp <- rbind(ppp_2008_2016, ppp_2017, copy(ppp_2017)[, year := 2018])
ppp[state == "Washington", state := "WA"]
ppp[state == "California", state := "CA"]
  
fwrite(ppp, "../data_clean/state_ppp.csv")

Warning message in fread("../data_raw/SARPP_STATE_2008_2016.csv"):
“Stopped early on line 210. Expected 17 fields but found 1. Consider fill=TRUE and comment.char=. First discarded non-empty line: <<"Note: See the included footnote file.">>”
Warning message in melt.data.table(ppp_2008_2016, id.vars = c("GeoName"), variable.name = "year"):
“'measure.vars' [GeoFIPS, Region, TableName, LineCode, ...] are not all of the same type. By order of hierarchy, the molten data value column will be of type 'character'. All measure variables not of type 'character' will be coerced too. Check DETAILS in ?melt.data.table for more on coercion.”
